In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import torch
from torch import nn
import random
import torchvision
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import json
from collections import namedtuple
from sklearn.metrics import classification_report
import cv2

In [3]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])
resize = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
#Tao cac class 2C, 3C, 4C
def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

#Them Noise vao dataset
# class AddGaussianNoise(object):
#     def __init__(self, mean=0., std=1.):
#         self.std = std
#         self.mean = mean
        
#     def __call__(self, tensor):
#         return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
#     def __repr__(self):
#         return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

#CHuan bi data truoc khi dua vao model
def prepare_data():
  transform_train = transforms.Compose([
      transforms.Resize((resize, resize)),
      #transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),  
      #transforms.RandomApply(transforms.GaussianBlur(kernel_size=501), p=0.5),                          
      transforms.ToTensor()
  ])

  transform_test = transforms.Compose([
    transforms.Resize((resize, resize)),
    #transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
    #transforms.RandomApply(transforms.GaussianBlur(kernel_size=501), p=0.5),
    transforms.ToTensor()
  ])
  #Tao duong dan den data
  trainset = torchvision.datasets.ImageFolder(root='./drive/MyDrive/data/DATA_CHAMBER_2021/train', transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root='./drive/MyDrive/data/DATA_CHAMBER_2021/test', transform=transform_test)

  #In ra so luong anh
  print("Number of Image in train set:", len(trainset))
  print("Number of Image in test set:", len(testset))
  print("Classes: ", trainset.class_to_idx)

  return TrainTest(train=trainset, test=testset)

#Load data dua vao model
def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=32, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=32, shuffle=False, num_workers=4)
  print("Num batch in train set: ", len(trainloader))
  return TrainTest(train=trainloader, test=testloader)


#Ham train model
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 20
  step = 0
  for images, labels in loader:
    step += 1
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if step % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {step} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
#Ham test
def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for images, labels in loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue



In [4]:
#Ham khoi tao model
def main(MODEL='vgg16'):
  classes = get_classes()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0")

  use_pretrained = False

  if MODEL == 'vgg19':
    PATH='./vgg19.pth'
    model = models.vgg19(pretrained=use_pretrained)
    model.classifier[6] = nn.Linear(in_features=4096, out_features=3)
  elif MODEL == 'resnet50':
    PATH='./resnet18.pth'
    model = models.resnet18(pretrained=use_pretrained)
    model.classifier[6] = nn.Linear(in_features=512, out_features=3)
  else:
    PATH='./vgg16.pth'
    model = models.vgg16(pretrained=use_pretrained)
    model.classifier[6] = nn.Linear(in_features=4096, out_features=3) 


  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(ytrue, ypred, target_names=classes))

    torch.save(model.state_dict(), PATH)

  return model

In [ ]:
#Model VGG16
model_VGG16 = main(MODEL='vgg16')

Number of Image in train set: 6717
Number of Image in test set: 1607
Classes:  {'2C': 0, '3C': 1, '4C': 2}
Num batch in train set:  210


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 0 step 19 ave_loss 1.0454
Epoch 0 step 39 ave_loss 1.0794
Epoch 0 step 59 ave_loss 1.0416
Epoch 0 step 79 ave_loss 0.7552
Epoch 0 step 99 ave_loss 0.5806
Epoch 0 step 119 ave_loss 0.5349
Epoch 0 step 139 ave_loss 0.4836
Epoch 0 step 159 ave_loss 0.2785
Epoch 0 step 179 ave_loss 0.2542
Epoch 0 step 199 ave_loss 0.1682
              precision    recall  f1-score   support

          2C       0.89      0.86      0.87       409
          3C       0.84      0.94      0.89       367
          4C       1.00      0.97      0.98       831

    accuracy                           0.93      1607
   macro avg       0.91      0.92      0.91      1607
weighted avg       0.93      0.93      0.93      1607

Epoch 1 step 19 ave_loss 0.1494
Epoch 1 step 39 ave_loss 0.1396
Epoch 1 step 59 ave_loss 0.1195
Epoch 1 step 79 ave_loss 0.1064
Epoch 1 step 99 ave_loss 0.1000
Epoch 1 step 119 ave_loss 0.1474
Epoch 1 step 139 ave_loss 0.0991
Epoch 1 step 159 ave_loss 0.8103
Epoch 1 step 179 ave_loss 0.4496
Ep

In [ ]:
#Model VGG19
model_VGG19 = main(MODEL='vgg19')

In [ ]:
#Model Resnet18
model_Resnet18 = main(MODEL='resnet18')